## SAR Calibration 

Create a graph to calibrate a Sentinel-1 GRD product

In [1]:
from snapista import Graph
from snapista import Operator, OperatorParams

Create a `Graph` object

In [2]:
g = Graph()

Add the `Read` node to the graph

In [3]:
path_to_manifest = 'path to manifest'

g.add_node(
    operator=Operator(
        "Read",
        formatName="SENTINEL-1",
        file=path_to_manifest,
    ),
    node_id="read",
)

Add the remaining nodes:

In [4]:
g.add_node(
        operator=Operator("Apply-Orbit-File", continueOnFail="true"),
        node_id="apply-orbit-file",
        source="read",
)

g.add_node(
    operator=Operator(
        "Remove-GRD-Border-Noise", borderLimit="2000", trimThreshold="0.2"
    ),
    node_id="noise-removal",
    source="apply-orbit-file",
)

g.add_node(
    operator=Operator("Calibration"),
    node_id="calibration",
    source="noise-removal",
)

g.add_node(
    operator=Operator("LinearToFromdB"),
    node_id="linear",
    source="calibration",
)

g.add_node(
    operator=Operator(
        "Terrain-Correction",
        pixelSpacingInMeter="20.0",
        demName="SRTM 3Sec",
        mapProjection="AUTO:42001",
        ),
        node_id="terrain-correction",
        source="linear",
)

g.add_node(
    operator=Operator("Write", file='result.tif', formatName="GeoTIFF-BigTIFF"),
    node_id="write",
    source="terrain-correction",
)

Inspect the graph

In [5]:
g.view()

<graph>
  <version>1.0</version>
  <node id="read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>path to manifest</file>
      <formatName>SENTINEL-1</formatName>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
    </parameters>
  </node>
  <node id="apply-orbit-file">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <continueOnFail>true</continueOnFail>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <polyDegree>3</polyDegree>
    </parameters>
  </node>
  <node id="noise-removal">
    <operator>Remove-GRD-Border-Noise</operator>
    <sources>
      <sourceProduct refid="apply-orbit-file"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <borderLimit>2000

Run the graph

In [ ]:
g.run()